In [14]:
import mysql.connector
import tkinter as tk
from tkinter import simpledialog, messagebox

class Note:
    def __init__(self, title, content):
        self.title = title
        self.content = content

    def edit_title(self, new_title):
        self.title = new_title

    def edit_content(self, new_content):
        self.content = new_content

class NoteManager:
    def __init__(self, db_cursor):
        self.db_cursor = db_cursor

    def add_note(self, title, content):
        query = "INSERT INTO notes (title, content) VALUES (%s, %s)"
        self.db_cursor.execute(query, (title, content))

    def delete_note_by_id(self, note_id):
        query = "DELETE FROM notes WHERE id = %s"
        self.db_cursor.execute(query, (note_id,))

    def edit_note_by_id(self, note_id, new_title, new_content):
        query = "UPDATE notes SET title = %s, content = %s WHERE id = %s"
        self.db_cursor.execute(query, (new_title, new_content, note_id))

    def get_all_notes(self):
        query = "SELECT * FROM notes"
        self.db_cursor.execute(query)
        return self.db_cursor.fetchall()

    def get_note_by_id(self, note_id):
        query = "SELECT * FROM notes WHERE id = %s"
        self.db_cursor.execute(query, (note_id,))
        return self.db_cursor.fetchone()

class UI:
    def __init__(self, root, note_manager):
        self.note_manager = note_manager
        self.root = root
        self.root.title("Note-Taking App")

        self.toolbar_frame = tk.Frame(self.root)
        self.toolbar_frame.pack(side=tk.TOP, fill=tk.X, padx=10, pady=10)

        self.add_note_button = tk.Button(self.toolbar_frame, text="Add Note", command=self.add_note)
        self.add_note_button.pack(side=tk.LEFT)

        self.delete_note_button = tk.Button(self.toolbar_frame, text="Delete Note", command=self.delete_note)
        self.delete_note_button.pack(side=tk.LEFT)

        self.edit_note_button = tk.Button(self.toolbar_frame, text="Edit Note", command=self.edit_note)
        self.edit_note_button.pack(side=tk.LEFT)

        self.notes_listbox = tk.Listbox(self.root)
        self.notes_listbox.pack(fill=tk.BOTH, expand=True, padx=10, pady=(0, 10))

        self.notes_listbox.bind("<Double-1>", self.show_note_content)

        self.update_notes_listbox()

    def add_note(self):
        title = simpledialog.askstring("Input", "Enter the note title:")
        content = simpledialog.askstring("Input", "Enter the note content:")
        if title and content:
            self.note_manager.add_note(title, content)
            self.update_notes_listbox()
            messagebox.showinfo("Success", "Note added successfully!")

    def delete_note(self):
        selected_index = self.notes_listbox.curselection()
        if selected_index:
            note_id = self.notes[selected_index[0]]['id']
            self.note_manager.delete_note_by_id(note_id)
            self.update_notes_listbox()
            messagebox.showinfo("Success", "Note deleted successfully!")
        else:
            messagebox.showwarning("Warning", "Please select a note to delete.")

    def edit_note(self):
        selected_index = self.notes_listbox.curselection()
        if selected_index:
            new_title = simpledialog.askstring("Input", "Enter the new title:")
            new_content = simpledialog.askstring("Input", "Enter the new content:")
            if new_title and new_content:
                note_id = self.notes[selected_index[0]]['id']
                self.note_manager.edit_note_by_id(note_id, new_title, new_content)
                self.update_notes_listbox()
                messagebox.showinfo("Success", "Note updated successfully!")
        else:
            messagebox.showwarning("Warning", "Please select a note to edit.")

    def show_note_content(self, event):
        selected_index = self.notes_listbox.curselection()
        if selected_index:
            note_id = self.notes[selected_index[0]]['id']
            note = self.note_manager.get_note_by_id(note_id)
            content_window = tk.Toplevel(self.root)
            content_window.title("Note Content")
            tk.Label(content_window, text=f"Title: {note['title']}", font=('Arial', 14, 'bold')).pack(pady=10)
            tk.Label(content_window, text=note['content'], wraplength=400, justify=tk.LEFT).pack(padx=10, pady=10)

    def update_notes_listbox(self):
        self.notes_listbox.delete(0, tk.END)
        self.notes = self.note_manager.get_all_notes()
        for idx, note in enumerate(self.notes):
            self.notes_listbox.insert(tk.END, f"Note {idx + 1}: {note['title']}")

# Main Application
if __name__ == "__main__":
    db_connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="admin",
        database="notes_app"
    )
    db_cursor = db_connection.cursor(dictionary=True)

    note_manager = NoteManager(db_cursor)
    root = tk.Tk()
    ui = UI(root, note_manager)
    root.mainloop()

    db_connection.commit()
    db_cursor.close()
    db_connection.close()


In [11]:
''';;

SyntaxError: incomplete input (854695169.py, line 1)